In [9]:
import os
import pandas as pd
import numpy as np

DATA_PATH = "../data/Unified Dataset/unified_dataset.csv"  # بدّلها إذا الاسم مختلف
OUT_PATH  = "../data/Unified Dataset/unified_dataset_cleaned.csv"

In [10]:
df = pd.read_csv(DATA_PATH, low_memory=False)
print("Shape:", df.shape)
df.head()

Shape: (225633, 61)


,age,class,country,diabetes,diagnosis,ethnicity,family_history,fti,fti_measured,ftimeasured,...,thyroidsurgery,thyroxine,tsh,tsh_level,tsh_measured,tshmeasured,tt4,tt4_measured,tt4measured,tumor
0,29,-,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,0.3,NaN,t,NaN,NaN,f,NaN,f
1,29,-,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,1.6,NaN,t,NaN,128.0,t,NaN,f
2,41,-,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,NaN,NaN,f,NaN,NaN,f,NaN,f
3,36,-,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,NaN,NaN,f,NaN,NaN,f,NaN,f
4,32,S,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,...,NaN,NaN,NaN,NaN,f,NaN,NaN,f,NaN,f


In [11]:
df.info()
missing = df.isna().mean().sort_values(ascending=False)
missing.head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225633 entries, 0 to 225632
Data columns (total 61 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   age                      225633 non-null  int64  
 1   class                    225633 non-null  object 
 2   country                  212691 non-null  object 
 3   diabetes                 212691 non-null  object 
 4   diagnosis                212691 non-null  object 
 5   ethnicity                212691 non-null  object 
 6   family_history           212691 non-null  object 
 7   fti                      12140 non-null   float64
 8   fti_measured             9172 non-null    object 
 9   ftimeasured              3770 non-null    object 
 10  gender                   212691 non-null  object 
 11  goitre                   12942 non-null   object 
 12  hypopituitary            12942 non-null   object 
 13  i131_treatment           9172 non-null    object 
 14  i131

ftimeasured                0.983291
onantithyroidmedication    0.983291
queryhyperthyroid          0.983291
i131treatment              0.983291
tbgmeasured                0.983291
t4umeasured                0.983291
thyroxine                  0.983291
thyroidsurgery             0.983291
t3measured                 0.983291
t4                         0.983291
referral source            0.983291
queryonthyroxine           0.983291
queryhypothyroid           0.983291
tt4measured                0.983291
tshmeasured                0.983291
tbg                        0.981745
tt4                        0.961309
thyroid_surgery            0.959350
referral_source            0.959350
t3_measured                0.959350
dtype: float64

In [12]:
pairs = [
    ("tsh_measured", "tshmeasured"),
    ("t3_measured", "t3measured"),
    ("tt4_measured", "tt4measured"),
    ("t4u_measured", "t4umeasured"),
    ("fti_measured", "ftimeasured"),
]

for a, b in pairs:
    if a in df.columns and b in df.columns:
        df[a] = df[a].combine_first(df[b])
        df.drop(columns=[b], inplace=True)
        print(f"Merged: {b} -> {a}")
    else:
        print(f"Skip (not found): {a}, {b}")

print("New shape:", df.shape)


Merged: tshmeasured -> tsh_measured
Merged: t3measured -> t3_measured
Merged: tt4measured -> tt4_measured
Merged: t4umeasured -> t4u_measured
Merged: ftimeasured -> fti_measured
New shape: (225633, 56)


In [13]:
df.to_csv(OUT_PATH, index=False)
print("Saved to:", OUT_PATH)


Saved to: ../data/Unified Dataset/unified_dataset_cleaned.csv


In [14]:
[c for c in df.columns if "sex" in c.lower() or "gender" in c.lower()]

['gender', 'sex']

In [15]:
import pandas as pd
import numpy as np

# 1) ندمجو gender داخل sex (إذا sex فيها NaN نعمرها من gender)
df["sex"] = df["sex"].combine_first(df["gender"])

# 2) نوحدو القيم: Male/Female -> M/F
df["sex"] = (
    df["sex"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({
        "male": "M",
        "m": "M",
        "female": "F",
        "f": "F",
        "nan": np.nan
    })
)

# 3) نحذف gender (خلاص ولاّت مدموجة)
df.drop(columns=["gender"], inplace=True)

# 4) check
print(df["sex"].value_counts(dropna=False))

sex
F      136077
M       89099
NaN       457
Name: count, dtype: int64


In [16]:
df["sex"].unique()

array(['F', 'M', nan], dtype=object)

In [17]:
[c for c in df.columns if "tsh" in c.lower()]

['tsh', 'tsh_level', 'tsh_measured']

In [18]:
[c for c in df.columns if "t3" in c.lower()]

['t3', 't3_level', 't3_measured']

In [19]:
[c for c in df.columns if "t4" in c.lower()]

['t4', 't4_level', 't4u', 't4u_measured', 'tt4', 'tt4_measured']

In [20]:
# نتأكد النوع
print(df["tsh"].dtype)
print(df["tsh_level"].dtype)

# دمج
df["tsh"] = df["tsh"].combine_first(df["tsh_level"])
df.drop(columns=["tsh_level"], inplace=True)

print("TSH merged")

float64
float64
TSH merged


In [21]:
df["t3"] = df["t3"].combine_first(df["t3_level"])
df.drop(columns=["t3_level"], inplace=True)

print("T3 merged")

T3 merged


In [22]:
df["t4"] = df["t4"].combine_first(df["t4_level"])
df.drop(columns=["t4_level"], inplace=True)

print("T4 merged")

T4 merged


In [23]:
print(df.shape)

(225633, 52)


In [24]:
# المسار وين تحب تخزن
output_path = "../data/Unified Dataset/unified_dataset_step1_cleaned.csv"

# حفظ
df.to_csv(output_path, index=False)

print("Dataset saved successfully ✅")
print("Location:", output_path)

Dataset saved successfully ✅
Location: ../data/Unified Dataset/unified_dataset_step1_cleaned.csv


In [25]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

cols = df.columns.tolist()

similar_cols = []

for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if similarity(cols[i], cols[j]) > 0.8:
            similar_cols.append((cols[i], cols[j]))

similar_cols[:20]

[('fti_measured', 't3_measured'),
 ('fti_measured', 't4u_measured'),
 ('fti_measured', 'tbg_measured'),
 ('fti_measured', 'tsh_measured'),
 ('fti_measured', 'tt4_measured'),
 ('i131_treatment', 'i131treatment'),
 ('on_thyroxine', 'thyroxine'),
 ('query_hyperthyroid', 'query_hypothyroid'),
 ('query_hyperthyroid', 'queryhyperthyroid'),
 ('query_hyperthyroid', 'queryhypothyroid'),
 ('query_hypothyroid', 'queryhyperthyroid'),
 ('query_hypothyroid', 'queryhypothyroid'),
 ('query_on_thyroxine', 'queryonthyroxine'),
 ('queryhyperthyroid', 'queryhypothyroid'),
 ('referral source', 'referral_source'),
 ('t3_measured', 't4u_measured'),
 ('t3_measured', 'tbg_measured'),
 ('t3_measured', 'tbgmeasured'),
 ('t3_measured', 'tsh_measured'),
 ('t3_measured', 'tt4_measured')]

In [26]:
df["on_thyroxine"].value_counts(dropna=False)
df["thyroxine"].value_counts(dropna=False)

thyroxine
NaN    221863
f        3307
t         463
Name: count, dtype: int64

In [27]:
print("on_thyroxine:")
print(df["on_thyroxine"].value_counts(dropna=False))

print("\nthyroxine:")
print(df["thyroxine"].value_counts(dropna=False))

on_thyroxine:
on_thyroxine
NaN    216461
f        7932
t        1240
Name: count, dtype: int64

thyroxine:
thyroxine
NaN    221863
f        3307
t         463
Name: count, dtype: int64


In [28]:
df["on_thyroxine"] = df["on_thyroxine"].combine_first(df["thyroxine"])
df.drop(columns=["thyroxine"], inplace=True)

print("Merged thyroxine -> on_thyroxine")
print(df["on_thyroxine"].value_counts(dropna=False))
print("New shape:", df.shape)

Merged thyroxine -> on_thyroxine
on_thyroxine
NaN    212691
f       11239
t        1703
Name: count, dtype: int64
New shape: (225633, 51)


In [29]:
pairs_to_merge = [
    ("referral_source", "referral source"),
    ("i131_treatment", "i131treatment"),
    ("query_hyperthyroid", "queryhyperthyroid"),
    ("query_hypothyroid", "queryhypothyroid"),
    ("query_on_thyroxine", "queryonthyroxine"),
    ("tbg_measured", "tbgmeasured"),
]

for keep, drop in pairs_to_merge:
    if keep in df.columns and drop in df.columns:
        df[keep] = df[keep].combine_first(df[drop])
        df.drop(columns=[drop], inplace=True)
        print(f"✅ Merged {drop} -> {keep}")
    else:
        print(f"⚠️ Skip (missing): {keep} / {drop}")

print("✅ Done. New shape:", df.shape)

✅ Merged referral source -> referral_source
✅ Merged i131treatment -> i131_treatment
✅ Merged queryhyperthyroid -> query_hyperthyroid
✅ Merged queryhypothyroid -> query_hypothyroid
✅ Merged queryonthyroxine -> query_on_thyroxine
✅ Merged tbgmeasured -> tbg_measured
✅ Done. New shape: (225633, 45)


In [30]:
[c for c in df.columns if " " in c]

[]

In [31]:
df.to_csv("../data/Unified Dataset/unified_dataset_step2_harmonized.csv", index=False)
print("Saved ✅")

Saved ✅


In [32]:
df.columns[df.columns.duplicated()]

Index([], dtype='object')

In [33]:
similar_content = []

for i in range(len(df.columns)):
    for j in range(i+1, len(df.columns)):
        col1 = df[df.columns[i]]
        col2 = df[df.columns[j]]
        
        if col1.dtype == col2.dtype:
            similarity = (col1 == col2).mean()
            if similarity > 0.95:
                similar_content.append((df.columns[i], df.columns[j], similarity))

similar_content

[]

In [34]:
print("Total columns:", len(df.columns))
print("Unique columns:", len(set(df.columns)))

Total columns: 45
Unique columns: 45


In [35]:
df.columns.tolist()

['age',
 'class',
 'country',
 'diabetes',
 'diagnosis',
 'ethnicity',
 'family_history',
 'fti',
 'fti_measured',
 'goitre',
 'hypopituitary',
 'i131_treatment',
 'iodine_deficiency',
 'lithium',
 'nodule_size',
 'obesity',
 'on_antithyroid_meds',
 'on_thyroxine',
 'onantithyroidmedication',
 'patient_id',
 'pregnant',
 'psych',
 'query_hyperthyroid',
 'query_hypothyroid',
 'query_on_thyroxine',
 'radiation_exposure',
 'referral_source',
 'sex',
 'sick',
 'smoking',
 'source',
 't3',
 't3_measured',
 't4',
 't4u',
 't4u_measured',
 'tbg',
 'tbg_measured',
 'thyroid_surgery',
 'thyroidsurgery',
 'tsh',
 'tsh_measured',
 'tt4',
 'tt4_measured',
 'tumor']

In [36]:
new_order = [

# Identifiers
"patient_id", "source",

# Demographics
"age", "sex", "country", "ethnicity",

# Risk factors
"family_history", "iodine_deficiency", "diabetes", "obesity",
"lithium", "psych", "pregnant",

# Clinical conditions
"goitre", "hypopituitary", "tumor", "nodule_size", "diagnosis",

# Treatment
"on_thyroxine", "on_antithyroid_meds",
"onantithyroidmedication", "i131_treatment",

# Lab values
"tsh", "t3", "tt4", "t4", "t4u", "fti",

# Measured flags
"tsh_measured", "t3_measured",
"tt4_measured", "t4u_measured", "fti_measured",

# Queries
"query_hyperthyroid", "query_hypothyroid",
"query_on_thyroxine",

# Target
"class"
]

# نحتافظ غير باللي موجودين فعلاً
new_order = [c for c in new_order if c in df.columns]

df = df[new_order]

print("Columns organized medically ✅")

Columns organized medically ✅


In [37]:
df.to_csv("../data/Unified Dataset/unified_dataset_step3_medical_order.csv", index=False)

In [38]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225633 entries, 0 to 225632
Data columns (total 37 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   patient_id               221863 non-null  float64
 1   source                   225633 non-null  object 
 2   age                      225633 non-null  int64  
 3   sex                      225176 non-null  object 
 4   country                  212691 non-null  object 
 5   ethnicity                212691 non-null  object 
 6   family_history           212691 non-null  object 
 7   iodine_deficiency        212691 non-null  object 
 8   diabetes                 212691 non-null  object 
 9   obesity                  212691 non-null  object 
 10  lithium                  12942 non-null   object 
 11  psych                    12942 non-null   object 
 12  pregnant                 12942 non-null   object 
 13  goitre                   12942 non-null   object 
 14  hypo

In [39]:
df.duplicated().sum()

np.int64(65)

In [40]:
df["patient_id"].duplicated().sum()

np.int64(3769)

In [41]:
df.groupby("patient_id")["source"].nunique().sort_values(ascending=False).head(10)

patient_id
870119035.0    1
1.0            1
2.0            1
3.0            1
4.0            1
5.0            1
6.0            1
870116045.0    1
870116044.0    1
870116043.0    1
Name: source, dtype: int64

In [42]:
print("Duplicate rows:", df.duplicated().sum())

Duplicate rows: 65


In [43]:
df = df.drop_duplicates()

print("New shape:", df.shape)
print("Duplicate rows after cleaning:", df.duplicated().sum())

New shape: (225568, 37)
Duplicate rows after cleaning: 0


In [44]:
print("Duplicate rows:", df.duplicated().sum())


Duplicate rows: 0


In [45]:
df["age"].describe()

count    225568.000000
mean         52.797179
std         239.708087
min           1.000000
25%          33.000000
50%          52.000000
75%          70.000000
max       65526.000000
Name: age, dtype: float64

In [46]:
df[df["age"] > 120]["age"].sort_values().head(20)

2976       455
10536      455
5710     65511
6392     65512
8105     65526
Name: age, dtype: int64

In [47]:
df[df["age"] > 100]["age"].sort_values().head(20)

2976       455
10536      455
5710     65511
6392     65512
8105     65526
Name: age, dtype: int64

In [48]:
df.loc[df["age"] > 100, "age"] = None

In [49]:
df["age"].describe()

count    225563.000000
mean         51.922944
std          21.488305
min           1.000000
25%          33.000000
50%          52.000000
75%          70.000000
max          97.000000
Name: age, dtype: float64

In [50]:
df[["tsh","t3","t4","tt4","fti","t4u"]].describe()

,tsh,t3,t4,tt4,fti,t4u
count,224726.000000,222964.000000,216396.000000,8730.000000,12075.000000,12068.000000
mean,5.045361,1.994643,9.876084,108.700305,109.730579,0.955257
std,6.209527,0.872285,13.655944,37.522670,42.830357,0.252386
min,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,2.300000,1.260000,6.390000,87.000000,92.000000,0.860000
50%,4.830000,2.000000,8.300000,104.000000,108.000000,0.960000
75%,7.430000,2.730000,10.210000,126.000000,126.000000,1.070000
max,530.000000,18.000000,430.000000,600.000000,881.000000,2.330000


In [51]:
labs = ["tsh","t3","t4","tt4","fti","t4u"]

summary = (
    df.groupby("source")[labs]
      .agg(["count","median","mean","std","min", lambda s: s.quantile(0.95), "max"])
)

summary

tsh                                                       \
              count median      mean        std    min <lambda_0>    max   
source                                                                     
cancer_risk  212691   5.04  5.045102   2.860264  0.100       9.51   10.0   
hypothyroid    3705   1.20  4.671165  23.541914  0.000      12.00  530.0   
thyroidDF      8330   1.40  5.218403  24.184006  0.005      14.00  530.0   

                 t3                   ...  fti                     t4u         \
              count median      mean  ...  min <lambda_0>    max count median   
source                                ...                                       
cancer_risk  212691    2.0  2.001727  ...  NaN        NaN    NaN     0    NaN   
hypothyroid    3705    1.8  1.630591  ...  0.0      164.0  395.0  3705   0.96   
thyroidDF      6568    1.9  1.970629  ...  1.4      173.0  881.0  8363   0.96   

                                                        
                 mean       std   min <lambda_0>   max  
source                                                  
cancer_risk       NaN       NaN   NaN        NaN   NaN  
hypothyroid  0.908309  0.337208  0.00       1.31  2.32  
thyroidDF    0.976056  0.200360  0.17       1.32  2.33  

[3 rows x 42 columns]

In [52]:
for col in ["tsh","t3","t4","tt4","fti","t4u"]:
    q01 = df[col].quantile(0.01)
    q99 = df[col].quantile(0.99)
    df[col] = df[col].clip(q01, q99)

In [53]:
df[(df["sex"] == "M") & (df["pregnant"] == "t")]

,patient_id,source,age,sex,country,ethnicity,family_history,iodine_deficiency,diabetes,obesity,...,fti,tsh_measured,t3_measured,tt4_measured,t4u_measured,fti_measured,query_hyperthyroid,query_hypothyroid,query_on_thyroxine,class


In [54]:
df[(df["tsh_measured"] == "f") & (df["tsh"].notna())]

,patient_id,source,age,sex,country,ethnicity,family_history,iodine_deficiency,diabetes,obesity,...,fti,tsh_measured,t3_measured,tt4_measured,t4u_measured,fti_measured,query_hyperthyroid,query_hypothyroid,query_on_thyroxine,class
9178,NaN,hypothyroid,59.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,78.0,f,f,t,t,t,f,f,f,P
9201,NaN,hypothyroid,67.0,M,NaN,NaN,NaN,NaN,NaN,NaN,...,102.0,f,f,t,t,t,f,f,f,P
9211,NaN,hypothyroid,64.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,f,f,f,f,f,f,f,f,P
9219,NaN,hypothyroid,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,155.0,f,t,t,t,t,f,f,f,P
9226,NaN,hypothyroid,53.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,f,t,f,f,f,f,f,f,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12872,NaN,hypothyroid,61.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,f,f,f,f,f,f,f,f,P
12891,NaN,hypothyroid,36.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,f,t,t,f,f,f,f,f,P
12893,NaN,hypothyroid,34.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,174.0,f,t,t,t,t,t,f,f,P
12932,NaN,hypothyroid,61.0,M,NaN,NaN,NaN,NaN,NaN,NaN,...,103.0,f,t,t,t,t,f,f,f,P


In [55]:
df[(df["t3_measured"] == "f") & (df["t3"].notna())]
df[(df["tt4_measured"] == "f") & (df["tt4"].notna())]
df[(df["fti_measured"] == "f") & (df["fti"].notna())]

,patient_id,source,age,sex,country,ethnicity,family_history,iodine_deficiency,diabetes,obesity,...,fti,tsh_measured,t3_measured,tt4_measured,t4u_measured,fti_measured,query_hyperthyroid,query_hypothyroid,query_on_thyroxine,class
9173,NaN,hypothyroid,20.0,M,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,t,t,t,f,f,f,f,f,P
9175,NaN,hypothyroid,70.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,t,t,t,f,f,f,f,f,P
9199,NaN,hypothyroid,25.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,t,t,t,f,f,f,f,f,P
9211,NaN,hypothyroid,64.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,f,f,f,f,f,f,f,f,P
9226,NaN,hypothyroid,53.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,f,t,f,f,f,f,f,f,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12867,NaN,hypothyroid,56.0,M,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,t,t,t,f,f,f,t,f,P
12872,NaN,hypothyroid,61.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,f,f,f,f,f,f,f,f,P
12891,NaN,hypothyroid,36.0,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,f,t,t,f,f,f,f,f,P
12916,NaN,hypothyroid,59.0,M,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,t,t,t,f,f,f,f,f,P


In [56]:
measurement_pairs = [
    ("tsh_measured", "tsh"),
    ("t3_measured", "t3"),
    ("tt4_measured", "tt4"),
    ("fti_measured", "fti"),
    ("t4u_measured", "t4u"),
]

for flag, value in measurement_pairs:
    if flag in df.columns and value in df.columns:
        df.loc[df[flag] == "f", value] = None

print("Measurement consistency fixed ✅")

Measurement consistency fixed ✅


In [57]:
df[(df["tsh_measured"] == "f") & (df["tsh"].notna())]

,patient_id,source,age,sex,country,ethnicity,family_history,iodine_deficiency,diabetes,obesity,...,fti,tsh_measured,t3_measured,tt4_measured,t4u_measured,fti_measured,query_hyperthyroid,query_hypothyroid,query_on_thyroxine,class


In [58]:
checks = {
  "TSH": df[(df["tsh_measured"]=="f") & (df["tsh"].notna())].shape[0],
  "T3" : df[(df["t3_measured"]=="f") & (df["t3"].notna())].shape[0],
  "TT4": df[(df["tt4_measured"]=="f") & (df["tt4"].notna())].shape[0],
  "FTI": df[(df["fti_measured"]=="f") & (df["fti"].notna())].shape[0],
  "T4U": df[(df["t4u_measured"]=="f") & (df["t4u"].notna())].shape[0],
}
checks


{'TSH': 0, 'T3': 0, 'TT4': 0, 'FTI': 0, 'T4U': 0}

In [59]:
df[(df["sex"]=="M") & (df["pregnant"].isin(["t", True, 1]))].shape[0]

0

In [60]:
neg = {}
for col in ["tsh","t3","t4","tt4","fti","t4u","nodule_size"]:
    if col in df.columns:
        neg[col] = df[df[col] < 0].shape[0]
neg

{'tsh': 0, 't3': 0, 't4': 0, 'tt4': 0, 'fti': 0, 't4u': 0, 'nodule_size': 0}

In [61]:
df[(df["on_thyroxine"]=="f") & (df["query_on_thyroxine"]=="t")].shape[0]

172

In [62]:
pd.crosstab(df["query_on_thyroxine"], df["on_thyroxine"], dropna=False)

on_thyroxine,f,t,NaN
query_on_thyroxine,,,
f,11002,1672,0
t,172,31,0
NaN,0,0,212691


In [63]:
pd.crosstab(df["diagnosis"], df["class"])

class,High,Low,Medium
diagnosis,,,
Benign,9564,92153,61479
Malignant,22339,16235,10921


In [64]:
output_path = "../data/Unified Dataset/unified_dataset_validated.csv"

df.to_csv(output_path, index=False)

print("Dataset saved successfully ✅")
print("Location:", output_path)

Dataset saved successfully ✅
Location: ../data/Unified Dataset/unified_dataset_validated.csv
